# Professor Adams - Financial Risk Management (FRE 6123) 
# Homework #3

### README

---
---
### Part 1 Set up Environment

In [2]:
import math
import pandas as pd
import numpy as np
import math
from dataclasses import dataclass
from typing import List, Tuple

---
---

### Part 2 Answer

#### Question 1 

In [ ]:
# Credit risk metrics for a 4-year bullet loan

principal = 50_000_000.0  # USD
ytm = 0.03                # annual compounding
T = 4                     # years
recovery = 0.60
annual_pod = 0.0175       # per-year probability of default (conditional hazard)

# (a) Maximum exposure (just before maturity), bullet structure
ead_max = principal * ((1 + ytm) ** T)

# (b) Maximum Loss Given Default
lgd_max = (1 - recovery) * ead_max

# (c) Anticipated default rate in Year 4 (marginal PD in the 4th year)
survival_to_year4 = (1 - annual_pod) ** 3
marginal_pd_year4 = annual_pod * survival_to_year4

print("Max Expected Exposure (EAD_max) =", format(ead_max, ".6f"))
print("Max LGD =", format(lgd_max, ".6f"))
print("Survival to start of Year 4 =", format(survival_to_year4, ".6f"))
print("Anticipated default rate in Year 4 =", format(marginal_pd_year4, ".6f"))


Max Expected Exposure (EAD_max) = 56275440.500000
Max LGD = 22510176.200000
Survival to start of Year 4 = 0.948413
Anticipated default rate in Year 4 = 0.016597


: 

#### Results (rounded)

* **$$ \mathrm{EAD}_{\max} \approx \text{56275440.50} \text{USD} $$**

* **$$ \mathrm{LGD}_{\max} \approx \text{22510176.20} \text{USD} $$**

* **Survival to start of Year 4 $\approx 0.948413 $**

* **Marginal PD in Year 4 $\approx 1.659700\% $**

#### Interpretation 

**(a) Maximum expected exposure (bullet loan)**
$$
\mathrm{EAD}_{\max} = \text{Principal} \times (1+\text{YTM})^{T} = 50\,\text{m} \times (1.03)^4 \approx \text{\$56.275440}\,\text{m}
$$

**(b) Maximum loss given default**
$$
\mathrm{LGD}_{\max} = (1-\text{Recovery}) \times \mathrm{EAD}_{\max} = 0.40 \times 56.275440\,\text{m} \approx \text{\$22.510176}\,\text{m}
$$

**(c) Anticipated default rate in Year 4 (marginal PD)**
$$
\Pr[\text{default in Year 4}] = \text{POD} \times (1-\text{POD})^3 = 0.0175 \times (1-0.0175)^3 \approx 0.016597\ (\text{i.e., }1.659700\%)
$$

---
---

In [ ]:
# CDS pricing using standard-coupon convention with Effective Duration (RPV01 proxy)
# Assumptions:
# - Univox is HY, so the standard running coupon C = 500 bps
# - EffDurCDS ~ RPV01 (in years) at the given time
# - Upfront ≈ (C - s) * EffDur * Notional   (seller perspective; buyer is opposite sign)
# - Ignore accrual/fees and curve shape (one-factor approximation)

notional = 100_000_000.0  # USD
s0 = 0.0375               # 375 bps at inception
C  = 0.0500               # 500 bps standard coupon for HY
EffDur_0 = 4.8            # at inception

# (a) Inception upfront (seller perspective; + means seller receives from buyer)
upfront_seller = (C - s0) * EffDur_0 * notional
upfront_buyer  = -upfront_seller

# (b) 1y later: spread widens to 575 bps, EffDur=3.75; MTM from seller perspective
s1 = 0.0575
EffDur_1 = 3.75
mtm_seller_t1 = (C - s1) * EffDur_1 * notional

# (c) 2y later: credit event; recovery 0.84 ⇒ seller pays (1 - R) * notional
R = 0.84
settlement = (1 - R) * notional

print("=== (a) Inception ===")
print(f"Seller upfront (USD): {upfront_seller:,.2f}")
print(f"Buyer upfront  (USD): {upfront_buyer:,.2f}")

print("\n=== (b) At t=1y (seller perspective) ===")
print(f"CDS MTM (USD): {mtm_seller_t1:,.2f}")

print("\n=== (c) Credit event at t=2y ===")
print(f"Settlement amount (seller pays, USD): {settlement:,.2f}")


=== (a) Inception ===
Seller upfront (USD): 6,000,000.00
Buyer upfront  (USD): -6,000,000.00

=== (b) At t=1y (seller perspective) ===
CDS MTM (USD): -2,812,500.00

=== (c) Credit event at t=2y ===
Settlement amount (seller pays, USD): 16,000,000.00


: 

: 